~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Agentic-Tx Demo with Hugging Face

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/TxGemma/[TxGemma]Agentic_Demo_with_Hugging_Face.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-gemini/gemma-cookbook/blob/main/TxGemma/%5BTxGemma%5DAgentic_Demo_with_Hugging_Face.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/txgemma-release-67dd92e931c857d15e4d1e87">
      <img alt="Hugging Face logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on Hugging Face
    </a>
  </td>
</tr></tbody></table>

This Colab notebook provides a basic demo of using Agentic-Tx, a therapeutics-focused LLM agent. Agentic-Tx builds on TxGemma, a collection of large language models built upon Gemma 2, that generates predictions, classifications or text based on therapeutic related data.

Learn more about TxGemma at [this page](https://developers.google.com/health-ai-developer-foundations/txgemma).

## Setup

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the TxGemma model. Choose an appropriate runtime when starting your Colab session.

You can try out TxGemma 2B and 9B* for free using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

*To run the demo with both TxGemma 2B predict and 9B chat models on a T4 GPU, use 4-bit quantization to reduce memory usage and speed up inference. Note that the performance of quantized versions has not been evaluated.

### Get access to TxGemma

Before you get started, make sure that you have access to TxGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [TxGemma model page](https://huggingface.co/google/txgemma-2b-predict) and accept the usage conditions.

### Configure your HF token and Gemini token

Add your Hugging Face & Gemini token to the Colab Secrets manager to securely store it.

1. Ensure you have a [Gemini API key](https://ai.google.dev/gemini-api/docs/api-key) and a [HF_Token](https://huggingface.co/docs/hub/en/security-tokens)
2. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>

3. Create two new secrets with the name `HF_TOKEN` and `GEMINI_API_KEY`.
4. Copy/paste your token key into the Value input box of `HF_TOKEN` and `GEMINI_API_KEY` .
5. Toggle the button on the left to allow notebook access to the secret.

In [ ]:
import os, re
from google.colab import userdata
import google.generativeai as genai
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

### Install dependencies

In [ ]:
! pip install --upgrade --quiet accelerate bitsandbytes huggingface_hub transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 53.6 MB/s eta 0:00:00


### Load prompt template

First, load a JSON file that contains the prompt format for various TDC tasks.

In [ ]:
import json
from huggingface_hub import hf_hub_download

tdc_prompts_filepath = hf_hub_download(
    repo_id="google/txgemma-2b-predict",
    filename="tdc_prompts.json",
)

with open(tdc_prompts_filepath, "r") as f:
    tdc_prompts_json = json.load(f)

tdc_prompts.json:   0%|          | 0.00/768k [00:00<?, ?B/s]

### Download the prediction and chat model from Hugging Face Hub

Here, we are going on HuggingFace to download and load both the prediction and chat version of TxGemma. These will later be transformed into tools.

You can select the variants to use for Agentic-Tx at the dropdown and whether you want to include a chat variant. Make sure your runtime has enough RAM to run all the models.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

PREDICT_VARIANT = "2b-predict"  # @param ["2b-predict", "9b-predict", "27b-predict"]
CHAT_VARIANT = "9b-chat" # @param ["9b-chat", "27b-chat"]
USE_CHAT = True # @param {type: "boolean"}

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

predict_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{PREDICT_VARIANT}")
predict_model = AutoModelForCausalLM.from_pretrained(
    f"google/txgemma-{PREDICT_VARIANT}",
    device_map="auto",
    quantization_config=quantization_config,
)

if USE_CHAT:
    chat_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{CHAT_VARIANT}")
    chat_model = AutoModelForCausalLM.from_pretrained(
        f"google/txgemma-{CHAT_VARIANT}",
        device_map="auto",
        quantization_config=quantization_config,
    )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### Run inference on a sample binary classification task

Let's first try making predictions with both the models to make sure they are working. We're just going to use a sample TDC task with a sample drug.

In [ ]:
## Example task and input
task_name = "BBB_Martins"
input_type = "{Drug SMILES}"
drug_smiles = "CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21"
TDC_PROMPT = tdc_prompts_json[task_name].replace(input_type, drug_smiles)

def txgemma_predict(prompt):
    input_ids = predict_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = predict_model.generate(**input_ids, max_new_tokens=8)
    return predict_tokenizer.decode(outputs[0], skip_special_tokens=True)

def txgemma_chat(prompt):
    input_ids = chat_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = chat_model.generate(**input_ids, max_new_tokens=32)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Prediction model response: {txgemma_predict(TDC_PROMPT)}")
if USE_CHAT: print(f"Chat model response: {txgemma_chat(TDC_PROMPT)}")

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Prediction model response: Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:(B)
Chat model response: Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervo

### Making our first TxGemma tool (Chat)
We are now going to make a tool for our agent to use: a chat interface for our Gemini-based Agentic-Tx and TxGemma-Chat. This tool will allow our Agentic-Tx to ask TxGemma therapeutically relevant questions. We're going to provide some functionality to check if the tool was used (tool_is_used).

In [ ]:
# This will allow us to extract content from inside of ticks
def extract_prompt(text, word):
    code_block_pattern = rf"```{word}(.*?)```"
    code_blocks = re.findall(code_block_pattern, text, re.DOTALL)
    extracted_code = "\n".join(code_blocks).strip()
    return extracted_code

# This class will allow us to inferface with TxGemma
class TxGemmaChatTool:
    def __init__(self):
      self.tool_name = "Chat Tool"

    def use_tool(self, question):
        # Here, we are submitting a question to TxGemma
        response = txgemma_chat(question)
        return response

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```TxGemmaChat" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="TxGemmaChat")

    def instructions(self):
        # Here, we are **very** descriptively explaining how the tool works to the agent
        # This will be useful later on
        return (
            "=== Therapeutic Chat Tool Instructions ===\n"
            "### What This Tool Does\n"
            "The Therapeutic Chat Tool allows you to chat with a knowledgeable large language model named TxGemma trained on many therapeutics datasets."
            "### When and Why You Should Use It\n"
            "- If you have therapeutics related questions that you would benefit from asking TxGemma from.\n"
            "### How to Use It\n"
            "Format your query with triple backticks (```), and start with `TxGemmaChat`. Then on a new line:\n"
            "1) **Any question you would like to ask**\n\n"
            "Example:\n"
            "```TxGemmaChat\n"
            "What is a common drug used to treat ovarian cancer?\n"
            "```\n")

Let's now try out the tool by asking TxGemma-Chat a question.

In [ ]:
if USE_CHAT:
    chat_tool = TxGemmaChatTool()
    response = chat_tool.use_tool("Can Aspirin help with headaches? Yes or no?")
    print(response)

Can Aspirin help with headaches? Yes or no? 



### Making a TxGemma prediction (Clinical Toxicity)
We are now going to make a tool for our agent that allows them to predict whether a drug will be toxic in clinical trials. For this, we will interface with the predict version of TxGemma, which has strong predictive capabilities.


In [ ]:
# This class will allow us to predict toxicity using TxGemma
class ClinicalTox:
    def __init__(self):
      self.tool_name = "Clinical Toxicity Prediction"

    def use_tool(self, smiles_string):
        # Here, we are submitting the smiles to TxGemma, and returning the response
        prediction = txgemma_predict(tdc_prompts_json["ClinTox"].replace("{Drug SMILES}", smiles_string))
        if "(A)" in prediction:   prediction = f"{smiles_string} is not toxic!"
        elif "(B)" in prediction: prediction = f"{smiles_string} is toxic!"
        return prediction

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```ClinicalToxTool" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="ClinicalToxTool")

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
            "=== Clinical Toxicity Instructions ===\n"
            "The Clinical Toxicity Tool is designed to predict potential for toxicity for humans in clinicial trials.\n"
            "You can test the toxicity of different SMILES strings as they might affect humans.\n"
            "To properly use this tool, follow the format outlined below:\n"
            "1. **Form a clinical toxicity query**:\n"
            "```ClinicalToxTool\n<SMILES string here>\n```\n"
            "Example: ```ClinicalToxTool\nCN(C)C(=N)N=C(N)N\n```\n"
            "- Replace `<SMILES string here>` with an exact smiles string. "
            "A results will be returned to you describing the clinical toxicity.\n"
            "**Important Formatting Details**:\n"
            "- Use `ClinicalToxTool` as the exact keyword to begin your query.\n"
            "- Place your text after `ClinicalToxTool` on a new line.\n"
            "- Enclose the entire query using three backticks (```), as shown in the example above.\n")

Now we can test this out. The drug CC(=O)OC1=CC=CC=C1C(=O)O is a well known non-toxic drug, Aspirin. Let's test out the ClinicalTox to make sure the tool works correctly!

In [ ]:
# This is an example of gemini using the tool to predict toxicity
clintox = ClinicalTox()
prediction_aspirin = clintox.use_tool("CC(=O)OC1=CC=CC=C1C(=O)O")
print(prediction_aspirin)

CC(=O)OC1=CC=CC=C1C(=O)O is not toxic!


### Making a PubMed article search tool
We are now going to make a tool for our agent that allows them to search through PubMed articles. This allows our agent to have access to the latest biomedical research. We can access PubMed through the biopython API.


In [ ]:
! pip install --upgrade --quiet biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.9 MB/s eta 0:00:00


In [ ]:
from Bio import Medline, Entrez

# This class will allow us to interface with PubMed
class PubMedSearch:
    def __init__(self):
      self.tool_name = "PubMed Search"

    def tool_is_used(self, query: str):
        # This just checks to see if the tool call was evoked
        return "```PubMedSearch" in query

    def process_query(self, query: str):
        # Here, we clean to query to remove the tool call
        search_text = extract_prompt(query, word="PubMedSearch")
        return search_text.strip()

    def use_tool(self, search_text):
        # Here, we are searching through PubMed and returning relevant articles
        pmids = list()
        handle = Entrez.esearch(db="pubmed", sort="relevance", term=search_text, retmax=3)
        record = Entrez.read(handle)
        pmids = record.get("IdList", [])
        handle.close()

        if not pmids:
            return f"No PubMed articles found for '{search_text}' Please try a simpler search query."

        fetch_handle = Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="medline", retmode="text")
        records = list(Medline.parse(fetch_handle))
        fetch_handle.close()

        result_str = f"=== PubMed Search Results for: '{search_text}' ===\n"
        for i, record in enumerate(records, start=1):
            pmid = record.get("PMID", "N/A")
            title = record.get("TI", "No title available")
            abstract = record.get("AB", "No abstract available")
            journal = record.get("JT", "No journal info")
            pub_date = record.get("DP", "No date info")
            authors = record.get("AU", [])
            authors_str = ", ".join(authors[:3])
            result_str += (
                f"\n--- Article #{i} ---\n"
                f"PMID: {pmid}\n"
                f"Title: {title}\n"
                f"Authors: {authors_str}\n"
                f"Journal: {journal}\n"
                f"Publication Date: {pub_date}\n"
                f"Abstract: {abstract}\n")
        return f"Query: {search_text}\nResults: {result_str}"

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
            f"{'@' * 10}\n@@@ PubMed Search Tool Instructions @@@\n\n"
            "### What This Tool Does\n"
            "The PubMed Search Tool queries the NCBI Entrez API (PubMed) for a given search phrase, "
            "and retrieves metadata for a few of the top articles (PMID, title, authors, journal, date, abstract).\n\n"
            "### When / Why You Should Use It\n"
            "- To find **scientific literature** references on a specific biomedical topic.\n"
            "- To retrieve **abstracts, titles, authors**, and other metadata.\n\n"
            "### Query Format\n"
            "Wrap your request with triple backticks, starting with `PubMedSearch`. For example:\n\n"
            "```PubMedSearch\ncancer immunotherapy\n```\n\n"
            "### Example\n"
            "```PubMedSearch\nmachine learning in drug discovery\n```\n"
            "- This will search PubMed for articles related to 'machine learning in drug discovery', "
            "fetch up to 3 PMIDs, and return their titles, abstracts, etc.\n\n")

Let's now see how this tool works by getting the most up-to-date knowledge on drugs used for ovarian cancer.

In [ ]:
pubmed_tool = PubMedSearch()
search_results = pubmed_tool.use_tool("Drugs used for ovarian cancer")
print(search_results)

/usr/local/lib/python3.10/dist-packages/Bio/Entrez/__init__.py:734: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


Query: Drugs used for ovarian cancer
Results: === PubMed Search Results for: 'Drugs used for ovarian cancer' ===

--- Article #1 ---
PMID: 33123161
Title: Immunotherapy for Ovarian Cancer: Adjuvant, Combination, and Neoadjuvant.
Authors: Yang C, Xia BR, Zhang ZC
Journal: Frontiers in immunology
Publication Date: 2020
Abstract: Ovarian cancer is the most lethal gynecologic malignancy. Surgery and chemotherapy are the primary treatments for ovarian cancer; however, patients often succumb to recurrence with chemotherapeutic resistance within several years after the initial treatment. In the past two decades, immunotherapy has rapidly developed, and has revolutionized the treatment of various types of cancer. Despite the fact that immunotherapy response rates among ovarian cancer patients remain modest, treatment with immune checkpoint inhibitors (ICIs), chimeric antigen receptor (CAR)- and TCR-engineered T cells is rapidly developing. Therapeutic efficiency could be improved significantly

## Wrapping it all together

### Creating a tool manager
Now we need to package all of this together in a way that allows our agent to use these tools.

In [ ]:
# The tool manager will hold all of the tools, and provide an interface for the agent
class ToolManager:
    def __init__(self, toolset):
        self.toolset = toolset

    def tool_prompt(self):
        # This will let the agent know what tools it has access to
        tool_names = ", ".join([tool.tool_name for tool in self.toolset])
        return f"You have access to the following tools: {tool_names}\n{self.tool_instructions()}. You can only use one tool at a time. These are the only tools you have access to nothing else."

    def tool_instructions(self):
        # This allows the agent to know how to use the tools
        tool_instr = "\n".join([tool.instructions() for tool in self.toolset])
        return f"The following is a set of instructions on how to use each tool.\n{tool_instr}"

    def use_tool(self, query):
        # This will iterate through all of the tools
        # and find the correct tool that the agent requested
        for tool in self.toolset:
            if tool.tool_is_used(query):
                # use the tool and return the output
                return tool.use_tool(tool.process_query(query))
        return f"No tool match for search: {query}"

if USE_CHAT:
    tools = ToolManager([TxGemmaChatTool(), ClinicalTox(), PubMedSearch()])
else:
    tools = ToolManager([ClinicalTox(), PubMedSearch()])

### Creating a Gemini inference tool
The following tool will allow us to inference Gemini, which we can use to act as the agent orchestrator. Gemini will pick from the different tools we implemented as well as provide input to those tools in order to solve a problem. Let's make the inference structure.

In [ ]:
def inference_gemini(prompt, system_prompt, model_str):
  # Check to see that our model string matches
  if model_str == "gemini-2.0-pro":
    model = genai.GenerativeModel(model_name="gemini-2.0-pro-exp-02-05", system_instruction=system_prompt)
    response = model.generate_content(prompt)
    answer = response.text
  return answer

## Creating a therapeutics agent

Finally, we are going to create a Agentic-Tx with access to the three tools we made here, a therapeutics agent. We are going to ask it which drug is preferable for further development: a preferable drug, CC(=O)OC1=CC=CC=C1C(=O)O, versus a non-preferable drug, O=C(CCCCCCC(=O)Nc1ccccc1)NO.

In [ ]:
# This class defines our Agentic-Tx, wrapping together all of our tools and the orchestrator
class AgenticTx:
  def __init__(self, tool_manager, model_str, num_steps=5):
    self.curr_steps = 0
    self.num_steps = num_steps
    self.model_str = model_str
    self.tool_manager = tool_manager
    self.thoughts = list()
    self.actions  = list()
    self.observations = list()

  def reset(self):
    # Reset the number of steps taken
    self.curr_steps = 0

  def system_prompt(self, use_tools=True):
    # These are the system instructions for AgenticTx
    role_prompt = "You are an expert therapeutic agent. You answer accurately and thoroughly."
    prev_actions = f"You can perform a maximum of {self.num_steps} actions. You have performed {self.curr_steps} and have {self.num_steps - self.curr_steps - 1} left."
    if use_tools: tool_prompt = "You can use tools to solve problems and answer questions. " + self.tool_manager.tool_prompt()
    else: tool_prompt = "You cannot use any tools right now."
    return f"{role_prompt} {prev_actions} {tool_prompt}"

  def prior_information(self, query):
      info_txt = f"Question: {query}\n" if query is not None else ""
      for _i in range(self.curr_steps):
          info_txt += f"### Thought {_i + 1}: {self.thoughts[_i]}\n"
          info_txt += f"### Action {_i + 1}: {self.actions[_i]}\n"
          info_txt += f"### Observation {_i + 1}: {self.observations[_i]}\n\n"
          info_txt += "@"*20
      return info_txt

  def step(self, question):
    for _i in range(self.num_steps):
      if self.curr_steps == self.num_steps-1:
        return inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou must now provide an answer to this question {question}",
            system_prompt=self.system_prompt(use_tools=False))
      else:
        # Provide a thought step, planning for the model
        thought = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou cannot currently use tools but you can think about the problem and what tools you want to use. This was the question, think about plans for how to use tools to answer this {question}. Let's think step by step (respond with only 1-2 sentences).\nThought: ",
            system_prompt=self.system_prompt(use_tools=False))
        # Provide a took action for the model
        action = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\n{thought}\nNow you must use tools to answer the following user query [{question}], closely following the tool instructions. Tool",
            system_prompt=self.system_prompt(use_tools=True))
        obs = self.tool_manager.use_tool(action)

        print("Thought:", thought)
        print("Action:",  action)
        print("Observation:",  obs)

        self.thoughts.append(thought)
        self.actions.append(action)
        self.observations.append(obs)

        self.curr_steps += 1


agentictx = AgenticTx(tool_manager=tools, model_str="gemini-2.0-pro")
# The model should select CC(=O)OC1=CC=CC=C1C(=O)O because O=C(CCCCCCC(=O)Nc1ccccc1)NO is toxic
response = agentictx.step("Which of the following drugs is preferred for further development? 1. CC(=O)OC1=CC=CC=C1C(=O)O or 2. O=C(CCCCCCC(=O)Nc1ccccc1)NO")
print("\nFinal Response:", response)

Thought: I need to compare the two molecules based on drug-likeness criteria. I will likely want to evaluate properties such as molecular weight, LogP, and the presence of any problematic functional groups, potentially using a tool to calculate these properties.

Action: Okay, I will analyze the two molecules based on drug-likeness. Let's start with the first molecule, aspirin.

```ClinicalToxTool
CC(=O)OC1=CC=CC=C1C(=O)O
```

Observation: CC(=O)OC1=CC=CC=C1C(=O)O is not toxic!
Thought: ### Thought 2: Aspirin (molecule 1) is a well-known drug. I should now analyze the second molecule, considering factors like its size, potential metabolic liabilities, and any reactive groups, perhaps using a property calculator tool if available. I'll also look for any obvious red flags related to drug development.

Action: ### Action 2: Now, let's analyze the second molecule for toxicity.

```ClinicalToxTool
O=C(CCCCCCC(=O)Nc1ccccc1)NO
```

Observation: O=C(CCCCCCC(=O)Nc1ccccc1)NO is not toxic!
Though